In [ ]:
!pip uninstall -y newspaper3k lxml
!pip install --upgrade lxml lxml_html_clean
!pip install newspaper3k

Found existing installation: lxml 5.3.1
Uninstalling lxml-5.3.1:
  Successfully uninstalled lxml-5.3.1
  Using cached lxml-5.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.7 kB)
Using cached lxml-5.3.1-cp311-cp311-manylinux_2_28_x86_64.whl (5.0 MB)
  Using cached newspaper3k-0.2.8-py3-none-any.whl.metadata (11 kB)
  Using cached cssselect-1.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached tldextract-5.1.3-py3-none-any.whl.metadata (11 kB)
  Using cached feedfinder2-0.0.4.tar.gz (3.3 kB)
  Preparing metadata (setup.py) ... done
  Using cached jieba3k-0.35.1.zip (7.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached tinysegmenter-0.3.tar.gz (16 kB)
  Preparing metadata (setup.py) ... done
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached requests_file-2.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached newspaper3k-0.2.8-py3-none-any.whl 

In [ ]:
pip install newspaper3k

In [ ]:
!pip install GoogleNews
!pip install yahoo_fin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 0.8.0 req

In [ ]:
!python -m pip install --index-url=https://blpapi.bloomberg.com/repository/releases/python/simple/ blpapi

Looking in indexes: https://blpapi.bloomberg.com/repository/releases/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 10.5 MB/s eta 0:00:00


In [ ]:
import requests
import csv
import nltk
import spacy
import time
from bs4 import BeautifulSoup
from newspaper import Article
from nltk.tokenize import sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from GoogleNews import GoogleNews
from yahoo_fin import news

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('vader_lexicon')

# Initialize NLP tools
nlp = spacy.load("en_core_web_sm")
sia = SentimentIntensityAnalyzer()

# Keywords for Jio stock-related sentences
keywords = [""]# Enter Stock Specific keywords

def is_relevant(text):
    return any(keyword in text for keyword in keywords)

# Used to Classify text on the basis of sentiment and Labelling it
def classify_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    if score > 0.1:
        return "Positive"
    elif score < -0.1:
        return "Negative"
    else:
        return "Neutral"


# Scraping from NewsAPI
def get_newsapi_links(api_key, num_articles=1000):
    url = f"https://newsapi.org/v2/everything?q=Jio+stock&sortBy=publishedAt&language=en&apiKey={api_key}"
    try:
        response = requests.get(url)
        data = response.json()
        return [article["url"] for article in data.get("articles", [])[:num_articles]]
    except Exception as e:
        print(f"Error fetching NewsAPI links: {e}")
        return []

#Scraping From NewsDataIO
def get_newsdataio_links(api_key, num_articles=1000):
    url = f"https://newsdata.io/api/1/news?apikey={api_key}&q=Jio&language=en"
    try:
        response = requests.get(url)
        data = response.json()
        return [article["link"] for article in data.get("results", [])[:num_articles]]
    except Exception as e:
        print(f"Error fetching NewsData.io links: {e}")
        return []

# Scraping From Google News
def get_google_news_links(num_articles=1000):
    googlenews = GoogleNews(lang="en", period="7d")
    googlenews.search("Jio stock")
    results = googlenews.result()
    return [article["link"] for article in results[:num_articles]]


# Scraping From Yahoo Finance
def get_yahoo_finance_links(num_articles=1000):
    articles = news.get_yf_rss("RELIANCE.NS")  # Reliance Industries ticker symbol
    return [article["link"] for article in articles[:num_articles]]

# Scraping From Alpha Vantage Links
def get_alpha_vantage_links(api_key, num_articles=1000):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=RELIANCE.NS&apikey={api_key}"
    try:
        response = requests.get(url)
        data = response.json()
        return [article["url"] for article in data.get("feed", [])[:num_articles]]
    except Exception as e:
        print(f"Error fetching Alpha Vantage news: {e}")
        return []

#Scrpaing From Stock News
def get_stocknewsapi_links(num_articles=5000, batch_size=100):
    collected_urls = set()
    page = 1

    while len(collected_urls) < num_articles:
        url = f"https://stocknewsapi.com/api/v1?tickers=AXP&items={batch_size}&page={page}&token={STOCK_NEWS_API_KEY}"
        try:
            response = requests.get(url)
            data = response.json()
            new_links = {article["news_url"] for article in data.get("data", [])}
            collected_urls.update(new_links)

            print(f"📡 Page {page}: Collected {len(new_links)} new articles. Total: {len(collected_urls)}")
            page += 1

            if len(new_links) == 0:
                break

            time.sleep(1)
        except Exception as e:
            print(f"❌ Error fetching StockNewsAPI: {e}")
            break

    return list(collected_urls)

# Scraping text Over Here
def scrape_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text if is_relevant(article.title) else None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

# API Keys (Replace with actual credentials)
ALPHA_VANTAGE_KEY = "8HOXVJD6FABJBNUQ"
NEWSAPI_KEY = "a2bcdb25ffb4404ea911aa363e05b48c"
STOCK_NEWS_API_KEY = "xvhgrwydlxpcg5pr1isujdpysapupc0s1sva8w9f"
NEWSDATAIO_KEY= "pub_68826b6cd23d3d89ba30ec65d6472fd96b09c"

# Collecting stock news URLs
urls = set(
    get_newsapi_links(NEWSAPI_KEY, 1000) +
    get_newsdataio_links(NEWSDATAIO_KEY, 1000) +
    get_google_news_links(1000) +
    get_yahoo_finance_links(1000) +
    get_alpha_vantage_links(ALPHA_VANTAGE_KEY, 1000) +
    get_stocknewsapi_links(5000)
)
print(f"\U0001F4E1 Total Jio News URLs Collected: {len(urls)}")

# Collecting sentences by class
positive_sentences, negative_sentences, neutral_sentences = [], [], []

for url in urls:
    text = scrape_text(url)
    if text:
        extracted_sentences = sent_tokenize(text)
        for sentence in extracted_sentences:
            if is_relevant(sentence):
                sentiment = classify_sentiment(sentence)
                if sentiment == "Positive" and len(positive_sentences) < 10000:
                    positive_sentences.append(sentence)
                elif sentiment == "Negative" and len(negative_sentences) < 10000:
                    negative_sentences.append(sentence)
                elif sentiment == "Neutral" and len(neutral_sentences) < 10000:
                    neutral_sentences.append(sentence)
    if len(positive_sentences) >= 10000 and len(negative_sentences) >= 10000 and len(neutral_sentences) >= 10000:
        break

# Save data to CSV
with open("stock_sentiment_data.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Sentence", "Sentiment"])
    writer.writerows([[s, "Positive"] for s in positive_sentences])
    writer.writerows([[s, "Negative"] for s in negative_sentences])
    writer.writerows([[s, "Neutral"] for s in neutral_sentences])

print(f"\u2705 Scraped & labeled Jio stock news: {len(positive_sentences)} Positive, {len(negative_sentences)} Negative, {len(neutral_sentences)} Neutral!")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


📡 Page 1: Collected 0 new articles. Total: 0
📡 Total Jio News URLs Collected: 69
Error fetching https://economictimes.indiatimes.com/markets/stocks/stock-liveblog/jio-financial-services-share-price-live-updates-25-feb-2025/liveblog/118543911.cms&ved=2ahUKEwiJ7Nvo1t-LAxVCD0QIHUssDV0QxfQBegQIBxAC&usg=AOvVaw2g4Vo8O5NPvlvxiuAePbR9: Article `download()` failed with 404 Client Error: Not Found for url: https://economictimes.indiatimes.com/markets/stocks/stock-liveblog/jio-financial-services-share-price-live-updates-25-feb-2025/liveblog/118543911.cms&ved=2ahUKEwiJ7Nvo1t-LAxVCD0QIHUssDV0QxfQBegQIBxAC&usg=AOvVaw2g4Vo8O5NPvlvxiuAePbR9 on URL https://economictimes.indiatimes.com/markets/stocks/stock-liveblog/jio-financial-services-share-price-live-updates-25-feb-2025/liveblog/118543911.cms&ved=2ahUKEwiJ7Nvo1t-LAxVCD0QIHUssDV0QxfQBegQIBxAC&usg=AOvVaw2g4Vo8O5NPvlvxiuAePbR9
Error fetching https://www.livemint.com/market/live-blog/jio-financial-services-share-price-today-latest-live-updates-on-25-feb